In [ ]:
#Setting the needed imports and path
import os
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

sns.set_theme(style="whitegrid")
DATA_PATH = Path("../data/Motor_Vehicle_Collisions_-_Crashes.csv")
OUT_DIR = Path("../data/processed")
OUT_DIR.mkdir(parents=True, exist_ok=True)

In [ ]:
def column_inventory(df, top_n=8):
    n = len(df)
    rows = []
    for col in df.columns:
        ser = df[col]
        nn = int(ser.notna().sum())
        null_rate = 1 - nn / n
        nunique = int(ser.nunique(dropna=True))
        top_vals = ser.dropna().value_counts().nlargest(top_n).to_dict()
        rows.append({
            "column": col,
            "dtype": str(ser.dtype),
            "n_nonnull": nn,
            "null_rate": round(float(null_rate), 4),
            "nunique": nunique,
            "top_vals_sample": top_vals
        })
    return pd.DataFrame(rows).sort_values("null_rate", ascending=False)

sample = pd.read_csv(DATA_PATH, nrows=5000, low_memory=False)
inv_df = column_inventory(sample)
inv_df.head(50) 
inv_df.to_csv(OUT_DIR / "column_inventory_sample.csv", index=False)
print("Inventory saved to:", OUT_DIR / "column_inventory_sample.csv")


In [ ]:
#The cols we're interested in
KEEP_COLS = [
    "CRASH DATE", "CRASH TIME",
    "BOROUGH", "ZIP CODE", "LATITUDE", "LONGITUDE", "LOCATION",
    "ON STREET NAME", "CROSS STREET NAME", "OFF STREET NAME",
    "NUMBER OF PERSONS INJURED", "NUMBER OF PERSONS KILLED",
    "NUMBER OF PEDESTRIANS INJURED", "NUMBER OF PEDESTRIANS KILLED",
    "NUMBER OF CYCLIST INJURED", "NUMBER OF CYCLIST KILLED",
    "NUMBER OF MOTORIST INJURED", "NUMBER OF MOTORIST KILLED",
    "CONTRIBUTING FACTOR VEHICLE 1",
    "VEHICLE TYPE CODE 1",
    "COLLISION_ID"
]

sample_cols = pd.read_csv(DATA_PATH, nrows=2).columns.tolist()
KEEP_COLS = [c for c in KEEP_COLS if c in sample_cols]
print("Final keep columns ({}):".format(len(KEEP_COLS)))
for c in KEEP_COLS:
    print(" -", c)

In [ ]:
df = pd.read_csv(DATA_PATH, usecols=KEEP_COLS, low_memory=False)
print("Loaded df shape:", df.shape)
df.head()

In [ ]:
#Cleaning the data

df.columns = [c.strip() for c in df.columns]

# ZIP CODE to string  
if "ZIP CODE" in df.columns:
    df["ZIP CODE"] = df["ZIP CODE"].astype(str).str.strip().replace({'nan': None, 'None': None})

# Ensuring numeric injury/killed columns are numeric 
num_cols = [c for c in df.columns if ("NUMBER OF" in c) or ("KILLED" in c) or ("INJURED" in c)]
for col in num_cols:
    df[col] = pd.to_numeric(df[col], errors="coerce").fillna(0).astype(int)

print("Numeric columns cleaned:", num_cols)
df[num_cols].describe().T

In [ ]:
# CRASH DATE is MM/DD/YYYY in this dataset (US). Time is mostly 'H:MM' or 'HH:MM'.
# Vectorized time normalization:
time_raw = df["CRASH TIME"].astype(str).str.strip().fillna("")
time_raw = time_raw.replace({"nan": "", "NaN": "", "None": ""})

# Extract hh:mm where present
hhmm = time_raw.str.extract(r'(?P<hhmm>\d{1,2}:\d{2})')['hhmm']

mask_no_colon = hhmm.isna()
digits = time_raw[mask_no_colon].str.extract(r'(\d{1,4})')[0].fillna('')
digits_padded = digits.str.zfill(4)
formatted = digits_padded.str.slice(0,2) + ':' + digits_padded.str.slice(2,4)
hhmm.loc[mask_no_colon] = formatted.values
df["time_norm"] = hhmm.fillna('').astype(str)

# Parse date explicitly as MM/DD/YYYY
df["date_parsed"] = pd.to_datetime(df["CRASH DATE"], format="%m/%d/%Y", errors="coerce")

# Combine date + normalized time
combined = df["date_parsed"].dt.strftime("%Y-%m-%d").fillna('') + ' ' + df["time_norm"].fillna('')
combined = combined.str.strip()
df["ts"] = pd.to_datetime(combined, errors="coerce", infer_datetime_format=True)

# Report success rate
total = len(df)
parsed = df["ts"].notna().sum()
print(f"Total rows: {total:,}")
print(f"Parsed ts: {parsed:,}  ({parsed/total:.2%})")

In [ ]:
mask_date_present_but_no_ts = df["CRASH DATE"].notna() & df["ts"].isna()
print("Rows where CRASH DATE present but ts failed:", mask_date_present_but_no_ts.sum())

# Show a small sample of problematic pairs to see patterns
display(df.loc[mask_date_present_but_no_ts, ["CRASH DATE", "CRASH TIME"]].drop_duplicates().head(40))

# Rows with no CRASH DATE (rare?) — they cannot be used for time analysis
print("Rows missing CRASH DATE entirely:", df["CRASH DATE"].isna().sum())

In [ ]:
# Keeping only rows with ts for time based analysis
df_time = df[df["ts"].notna()].copy()
df_time["hour"] = df_time["ts"].dt.hour
df_time["weekday"] = df_time["ts"].dt.dayofweek   
df_time["is_weekend"] = df_time["weekday"].isin([5,6]).astype(int)
df_time["date"] = df_time["ts"].dt.date

df_time["severe"] = (
    (df_time["NUMBER OF PERSONS INJURED"] > 0) |
    (df_time["NUMBER OF PERSONS KILLED"] > 0)
).astype(int)

print("df_time rows:", len(df_time))
df_time[["ts","hour","weekday","severe"]].head()

In [ ]:
df_geo = df_time.dropna(subset=["LATITUDE", "LONGITUDE"]).copy()

zero_coords_mask = (df_geo["LATITUDE"] == 0) | (df_geo["LONGITUDE"] == 0)
print("Rows with LAT or LON exactly 0:", zero_coords_mask.sum())

lat_min, lat_max = 40.4774, 40.9176
lon_min, lon_max = -74.2591, -73.7004

mask_valid_nyc = (
    df_geo["LATITUDE"].between(lat_min, lat_max) &
    df_geo["LONGITUDE"].between(lon_min, lon_max)
)
df_geo_clean = df_geo[mask_valid_nyc].copy()
print("Rows with geo coords before filter:", len(df_geo))
print("Rows inside NYC bounding box:", len(df_geo_clean))
print("Rows removed:", len(df_geo) - len(df_geo_clean))

PRECISION = 4   
df_geo_clean["lat_round"] = df_geo_clean["LATITUDE"].round(PRECISION)
df_geo_clean["lon_round"] = df_geo_clean["LONGITUDE"].round(PRECISION)
df_geo_clean["segment_id"] = df_geo_clean["lat_round"].astype(str) + "_" + df_geo_clean["lon_round"].astype(str)

print("Unique segments (approx):", df_geo_clean["segment_id"].nunique())
df_geo_clean[["LATITUDE","LONGITUDE","segment_id"]].head()


In [ ]:
#Sorting by day, hour and week + day
daily = df_time.set_index("ts").resample("D").size().rename("crash_count").reset_index()

hourly = df_time.groupby("hour").size().reindex(range(24), fill_value=0)

pivot = df_time.pivot_table(index="weekday", columns="hour", values="ts", aggfunc="count").fillna(0)

# Useing cleaned data to compute top segments 
top_segments = df_geo_clean["segment_id"].value_counts().nlargest(20)

print("Daily rows:", len(daily))
print("Hourly counts sum (sanity):", hourly.sum())
print("Top segments (cleaned):")
display(top_segments.head(20))

In [ ]:
#Plot for crashes per day

plt.figure(figsize=(14,4))
plt.plot(daily["ts"], daily["crash_count"], linewidth=1)
plt.title("Crashes per day NYC — Time series")
plt.xlabel("Date")
plt.ylabel("Crash count")
plt.tight_layout()
plt.show()

In [ ]:
#Plot for crashes per hour

hourly = df_time.groupby("hour").size().reindex(range(24), fill_value=0)

plt.figure(figsize=(10,4))
plt.bar(hourly.index, hourly.values, color='tab:red')
plt.xticks(range(24))
plt.xlabel("Hour of day")
plt.ylabel("Crash count")
plt.title("Crashes by hour of day")
plt.tight_layout()
plt.show()


In [ ]:
#Week x hour heatmap

plt.figure(figsize=(14,5))
sns.heatmap(pivot, cmap="Reds", linewidths=0.2)
plt.title("Crash counts: weekday (0=Mon) vs hour")
plt.xlabel("Hour")
plt.ylabel("Weekday (0=Mon)")
plt.tight_layout()
plt.show()

In [ ]:
#Top segments

top_segments = df_geo_clean["segment_id"].value_counts().nlargest(20)

fig, ax = plt.subplots(figsize=(10,6))
y = range(len(top_segments))
ax.barh(y, top_segments.values[::-1], color='tab:blue') 
ax.set_yticks(y)
ax.set_yticklabels(top_segments.index[::-1])
ax.set_xlabel("Crash count")
ax.set_title("Top 20 segments by crash count NYC filtered")
plt.tight_layout()
plt.show()

In [ ]:
#Saving the cleaned data

df_small = df_time.copy()

if "COLLISION_ID" in df_small.columns and "COLLISION_ID" in df_geo_clean.columns:
    df_small = df_small.merge(
        df_geo_clean[["COLLISION_ID", "segment_id"]],
        how="left",
        on="COLLISION_ID"
    )

cols_to_save = [
    "ts","date","hour","weekday","is_weekend","severe",
    "LATITUDE","LONGITUDE","segment_id",
    "BOROUGH","ZIP CODE","ON STREET NAME","CROSS STREET NAME","COLLISION_ID"
]
cols_to_save = [c for c in cols_to_save if c in df_small.columns]

out_file = OUT_DIR / "vehicle_collisions_subset.parquet"

try:
    df_small[cols_to_save].to_parquet(out_file, index=False)
    print("Saved cleaned subset to:", out_file)
except Exception as e:
    print("Failed to write parquet. If the error mentions 'pyarrow' or 'fastparquet',")
    print("install pyarrow in your active conda env and re-run this cell:")
    print("  conda install -c conda-forge pyarrow -y")
    print("\nFull error below:\n")
    raise
